In [1]:
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torchvision import transforms
import torch
from tqdm import tqdm
from pathlib import Path
from utils import get_logger, AverageMeter, ProgressMeter
import datetime
from moco_model import MoCo
import torchvision
from torch import nn
from dataset import *
import time
from train import cal_accuracy_top_k
import shutil

In [35]:
class DatasetForComputingStatistics(Dataset):
    def __init__(self, data, transform=None) -> None:
        super().__init__()
        self.data = data
        self.transform = transform
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        image = Image.open(self.data[index]).convert('L')
        if self.transform is not None:
            image = self.transform(image)
        return image
    
augs = transforms.Compose([transforms.Resize((200, 200)), transforms.ToTensor(), transforms.Normalize(mean=0, std=1)])


In [ ]:

OBSERVED_IAMGE_PATH = '../datasets/NDI_images/Integreted/Observed/'
observed_images = list(map(str, Path(OBSERVED_IAMGE_PATH).glob('*.jpg')))
EXTRA_IMAGE_PATH = '../datasets/NDI_images/Integreted/extra observed/cropped/'
observed_images.extend(list(map(str, Path(EXTRA_IMAGE_PATH).glob('*.jpg'))))
CALCULATED_IMAGE_PATH = '../datasets/NDI_images/Integreted/Calculated/'
calculated_images = list(map(str, Path(CALCULATED_IMAGE_PATH).glob('*.jpg')))

observed_datasets = DatasetForComputingStatistics(observed_images, augs)
calculated_datasets = DatasetForComputingStatistics(calculated_images, augs)

observed_dataloader = DataLoader(observed_datasets, batch_size=1)
calculated_dataloader = DataLoader(calculated_datasets, batch_size=1)

pixel_sum_observed = torch.tensor([0.])
pixel_sum_sq_observed = torch.tensor([0.])
pixel_sum_calculated = torch.tensor([0.])
pixel_sum_sq_calculated = torch.tensor([0.])

for inputs in tqdm(observed_dataloader):
    pixel_sum_observed += inputs.sum(axis=[0, 2, 3])
    pixel_sum_sq_observed += (inputs ** 2).sum(axis=[0, 2, 3])

for inputs in tqdm(calculated_dataloader):
    pixel_sum_calculated += inputs.sum(axis=[0, 2, 3])
    pixel_sum_sq_calculated += (inputs ** 2).sum(axis=[0, 2, 3])

pixel_count_observed = len(observed_dataloader) * 200 * 200
pixel_count_calculated = len(calculated_dataloader) * 200 * 200

total_mean_observed = pixel_sum_observed / pixel_count_observed
total_std_observed = torch.sqrt(pixel_sum_sq_observed / pixel_count_observed - total_mean_observed ** 2)

total_mean_calculated = pixel_sum_calculated / pixel_count_calculated
total_std_calculated = torch.sqrt(pixel_sum_sq_calculated / pixel_count_calculated - total_mean_calculated ** 2)

print(total_mean_observed, '\n', total_std_observed, '\n', total_mean_calculated, '\n', total_std_calculated)

In [39]:
OBSERVED_IAMGE_PATH = '../datasets/NDI_images/Integreted/Observed/'
all_images = list(map(str, Path(OBSERVED_IAMGE_PATH).glob('*.jpg')))
EXTRA_IMAGE_PATH = '../datasets/NDI_images/Integreted/extra observed/cropped/'
all_images.extend(list(map(str, Path(EXTRA_IMAGE_PATH).glob('*.jpg'))))
CALCULATED_IMAGE_PATH = '../datasets/NDI_images/Integreted/Calculated/'
all_images.extend(list(map(str, Path(CALCULATED_IMAGE_PATH).glob('*.jpg'))))

all_images_datasets = DatasetForComputingStatistics(all_images, augs)

all_images_dataloader = DataLoader(all_images_datasets, batch_size=1)

pixel_sum = torch.tensor([0.])
pixel_sum_sq = torch.tensor([0.])

for inputs in tqdm(all_images_dataloader):
    pixel_sum += inputs.sum(axis=[0, 2, 3])
    pixel_sum_sq += (inputs ** 2).sum(axis=[0, 2, 3])

pixel_count_all = len(all_images_dataloader) * 200 * 200

total_mean_all = pixel_sum / pixel_count_all
total_std_all = torch.sqrt(pixel_sum_sq / pixel_count_all - total_mean_all ** 2)

print(total_mean_all, '\n', total_std_all,)

100%|██████████| 4701/4701 [08:58<00:00,  8.72it/s]

tensor([0.0877]) 
 tensor([0.0850])


In [2]:
def train(train_loader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter('Time', ':6.3f')
    data_time = AverageMeter('Data', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    
    progress = ProgressMeter(len(train_loader), [batch_time, data_time, losses, top1, top5], prefix=f'Epoch: [{epoch}]')
    
    model.train()
    
    start = time.time()
    
    for i, images in enumerate(train_loader):
        data_time.update(time.time() - start)
        x1, x2 = torch.split(images, [1, 1], dim=1)
        x1 = x1.cuda()
        x2 = x2.cuda()
        output, target = model(im_q=x1, im_k=x2)
        loss = criterion(output, target)
        
        acc1, acc5 = cal_accuracy_top_k(output, target, top_k=(1, 5))
        losses.update(loss.item(), x1.size(0))
        top1.update(acc1, x1.size(0))
        top5.update(acc5, x1.size(0))
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        batch_time.update(time.time() - start)
        start = time.time()
        
        if i % 10 == 0:
            progress.display(i)
    return losses.avg, top1.avg, top5.avg

def save_checkpoint(state, filename='checkpoint.pth'):
    torch.save(state, './checkpoints/' + filename)

## Pretraining for Observed Images

In [ ]:
moco_dim = 512
moco_K = 1024
moco_m = 0.999
moco_T = 0.2
lr = 0.005
SGD_m = 0.9
SGD_wd = 0.0001

observed_mean = 0.086
observed_std = 0.0854

input_size = 200
batch_size = 64
Epochs = 300

In [19]:


OBSERVED_IAMGE_PATH = '../datasets/NDI_images/Integreted/Observed/'
observed_images = list(map(str, Path(OBSERVED_IAMGE_PATH).glob('*.jpg')))
EXTRA_IMAGE_PATH = '../datasets/NDI_images/Integreted/extra observed/cropped/'
observed_images.extend(list(map(str, Path(EXTRA_IMAGE_PATH).glob('*.jpg'))))

device = torch.device('cuda')
logger = get_logger(f'CL_for_pretraining.log')

model = torchvision.models.resnet50
model = MoCo(model, dim=moco_dim, K=moco_K, m=moco_m, T=moco_T, mlp=True, weights=None)
model.cuda()

criterion = nn.CrossEntropyLoss().cuda(device)
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=SGD_m, weight_decay=SGD_wd)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, Epochs)

normalize = transforms.Normalize(mean=observed_mean, std=observed_std)
augmentation = transforms.Compose([transforms.Grayscale(3), transforms.RandomApply([transforms.RandomRotation(180)], p=0.5),
                                   transforms.RandomResizedCrop(input_size, scale=(0.2, 1.)), transforms.ColorJitter(0.4, 0.4, 0.4, 0.1),
                                   transforms.RandomApply([GaussianBlur([.1, 2.])], p=0.5), transforms.Grayscale(1),
                                   transforms.RandomHorizontalFlip(), transforms.RandomVerticalFlip(), transforms.ToTensor(),
                                   GaussNoise(p=0.5), normalize])
observed_datasets = NDIDatasetForPretraining(observed_images, augmentation)
observed_dataloader = DataLoader(observed_datasets, batch_size=batch_size, shuffle=True, drop_last=True)

best_score = 0
for epoch in range(Epochs):
    loss, acc1, acc5 = train(observed_dataloader, model, criterion, optimizer, epoch)
    print(f'Epoch: {epoch}, loss {loss}, Acc@1 {acc1}, Acc@5 {acc5}, lr {scheduler.get_last_lr()}')
    scheduler.step()
    score = acc1 * 2.5 + acc5
    if score > best_score:
        best_score = score
        save_checkpoint({'epoch': epoch + 1, 'arch': 'ResNet50', 'state_dict': model.state_dict(), 
                        'optimizer': optimizer.state_dict(), 'scheduler': scheduler.state_dict(), 'norms': [observed_mean, observed_std]})

Epoch: [0][ 0/70]	Time  0.525 ( 0.525)	Data  0.443 ( 0.443)	Loss 2.3414e+00 (2.3414e+00)	Acc@1  64.00 ( 64.00)	Acc@5  64.00 ( 64.00)
Epoch: [0][10/70]	Time  0.528 ( 0.537)	Data  0.459 ( 0.467)	Loss 6.4680e+00 (5.4115e+00)	Acc@1   0.00 (  5.91)	Acc@5   1.00 (  6.36)
Epoch: [0][20/70]	Time  0.515 ( 0.533)	Data  0.446 ( 0.463)	Loss 6.8968e+00 (6.0760e+00)	Acc@1   2.00 (  3.33)	Acc@5   3.00 (  4.10)
Epoch: [0][30/70]	Time  0.543 ( 0.533)	Data  0.473 ( 0.463)	Loss 6.8749e+00 (6.3382e+00)	Acc@1   0.00 (  2.58)	Acc@5   0.00 (  3.68)
Epoch: [0][40/70]	Time  0.528 ( 0.534)	Data  0.458 ( 0.464)	Loss 6.8362e+00 (6.4637e+00)	Acc@1   3.00 (  2.44)	Acc@5   6.00 (  4.10)
Epoch: [0][50/70]	Time  0.492 ( 0.531)	Data  0.422 ( 0.461)	Loss 6.7904e+00 (6.5315e+00)	Acc@1   0.00 (  2.24)	Acc@5   4.00 (  4.10)
Epoch: [0][60/70]	Time  0.522 ( 0.530)	Data  0.452 ( 0.460)	Loss 6.7628e+00 (6.5716e+00)	Acc@1   1.00 (  1.98)	Acc@5   4.00 (  4.10)
Epoch: 0, Acc@1 1.9142857142857144, Acc@5 4.271428571428571
Epoch: [1

In [ ]:
start_epoch = Epochs
end_epoch = 300

for g in optimizer.param_groups:
    g['lr'] = lr / 10
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, end_epoch - start_epoch)

best_score = 0
for epoch in range(Epochs):
    loss, acc1, acc5 = train(observed_dataloader, model, criterion, optimizer, epoch)
    print(f'Epoch: {epoch}, loss {loss}, Acc@1 {acc1}, Acc@5 {acc5}, lr {scheduler.get_last_lr()}')
    scheduler.step()
    score = acc1 * 2.5 + acc5
    if score > best_score:
        best_score = score
        save_checkpoint({'epoch': epoch + 1, 'arch': 'ResNet50', 'state_dict': model.state_dict(), 
                        'optimizer': optimizer.state_dict(), 'scheduler': scheduler.state_dict(), 'norms': [observed_mean, observed_std]})

In [ ]:
# lr=0.015  200epochs acc@5 10 loss 6.1

## Pretaining on All images

In [3]:
moco_dim = 512
moco_K = 1024
moco_m = 0.999
moco_T = 0.2
lr = 0.015
SGD_m = 0.9
SGD_wd = 0.0001

all_mean = 0.0877
all_std = 0.085

input_size = 200
batch_size = 64
Epochs = 300

In [4]:
all_images = get_pretraining_image_list()

device = torch.device('cuda')
logger = get_logger(f'CL_for_pretraining.log')

model = torchvision.models.resnet50
model = MoCo(model, dim=moco_dim, K=moco_K, m=moco_m, T=moco_T, mlp=True, weights=None)
model.cuda()

criterion = nn.CrossEntropyLoss().cuda(device)
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=SGD_m, weight_decay=SGD_wd)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, Epochs)

normalize = transforms.Normalize(mean=all_mean, std=all_std)
augmentation = transforms.Compose([transforms.Grayscale(3), transforms.RandomApply([transforms.RandomRotation(180)], p=0.5),
                                   transforms.RandomResizedCrop(input_size, scale=(0.2, 1.)), transforms.ColorJitter(0.4, 0.4, 0.4, 0.1),
                                   transforms.RandomApply([GaussianBlur([.1, 2.])], p=0.5), transforms.Grayscale(1),
                                   transforms.RandomHorizontalFlip(), transforms.RandomVerticalFlip(), transforms.ToTensor(),
                                   GaussNoise(p=0.5), normalize])
all_images_datasets = NDIDatasetForPretraining(all_images, augmentation)
all_images_dataloader = DataLoader(all_images_datasets, batch_size=batch_size, shuffle=True, drop_last=True)

best_score = 0
for epoch in range(Epochs):
    loss, acc1, acc5 = train(all_images_dataloader, model, criterion, optimizer, epoch)
    print(f'Epoch: {epoch}, loss {loss}, Acc@1 {acc1}, Acc@5 {acc5}, lr {scheduler.get_last_lr()}')
    scheduler.step()
    score = acc1 * 2.5 + acc5
    if score > best_score:
        best_score = score
        save_checkpoint({'epoch': epoch + 1, 'arch': 'ResNet50', 'state_dict': model.state_dict(), 
                        'optimizer': optimizer.state_dict(), 'scheduler': scheduler.state_dict(), 'norms': [all_mean, all_std]})

Epoch: [0][ 0/73]	Time  2.106 ( 2.106)	Data  0.610 ( 0.610)	Loss 2.4706e+00 (2.4706e+00)	Acc@1  64.00 ( 64.00)	Acc@5  64.00 ( 64.00)
Epoch: [0][10/73]	Time  0.579 ( 0.801)	Data  0.509 ( 0.601)	Loss 6.4946e+00 (5.4151e+00)	Acc@1   0.00 (  5.82)	Acc@5   0.00 (  6.45)
Epoch: [0][20/73]	Time  0.553 ( 0.725)	Data  0.483 ( 0.587)	Loss 6.9161e+00 (6.0849e+00)	Acc@1   0.00 (  3.10)	Acc@5   0.00 (  3.57)
Epoch: [0][30/73]	Time  0.551 ( 0.711)	Data  0.481 ( 0.595)	Loss 6.8782e+00 (6.3495e+00)	Acc@1   1.00 (  2.19)	Acc@5   3.00 (  2.90)
Epoch: [0][40/73]	Time  0.575 ( 0.699)	Data  0.505 ( 0.595)	Loss 6.8954e+00 (6.4807e+00)	Acc@1   1.00 (  1.85)	Acc@5   2.00 (  2.88)
Epoch: [0][50/73]	Time  0.571 ( 0.690)	Data  0.503 ( 0.592)	Loss 6.8770e+00 (6.5576e+00)	Acc@1   1.00 (  1.57)	Acc@5   1.00 (  2.73)
Epoch: [0][60/73]	Time  0.703 ( 0.683)	Data  0.633 ( 0.590)	Loss 6.8403e+00 (6.6069e+00)	Acc@1   2.00 (  1.41)	Acc@5   3.00 (  2.67)
Epoch: [0][70/73]	Time  0.575 ( 0.680)	Data  0.505 ( 0.590)	Loss 6.84

NameError: name 'observed_mean' is not defined